In [1]:
import time
import os
import gc

import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.utils import shuffle, resample
from sklearn.metrics import mean_squared_error, r2_score
np.set_printoptions(precision=6, suppress=True)

import PIL
from PIL import Image

import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
tf.__version__

'2.3.0'

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

2 Physical GPUs, 2 Logical GPUs


In [3]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


# Hyperparameters

In [4]:
PRE_TRAINED = './models/pg_convnet.h5'
BEST_PATH = './models/pg_cv_transfer.h5'
TRAINING_EPOCHS = 200
LEARNING_RATE = 0.0015
EPSILON = 1e-08
BATCH_SIZE = 32

# Data preparation

In [5]:
l = np.load('./results/pg_dataset.npz', allow_pickle=True)
data_indices_2020S = l['data_indices_2020S']
output_label_2020S = l['output_label_2020S']
data_indices_2020W = l['data_indices_2020W']
output_label_2020W = l['output_label_2020W']
OUTPUT_MAXS = l['OUTPUT_MAXS']
OUTPUT_MINS = l['OUTPUT_MINS']

In [6]:
COLUMNS = ['node_len', 'node_dia', 'plant_h', 'leaf_area']

In [7]:
output_2020S_df = pd.DataFrame(output_label_2020S, index=data_indices_2020S, columns=COLUMNS)
output_2020W_df = pd.DataFrame(output_label_2020W, index=data_indices_2020W, columns=COLUMNS)

In [8]:
DIRECTORY = './images/2020_W/'
dir_list = os.listdir(DIRECTORY)
dataset_list = [direc for direc in dir_list if direc.startswith('LAI_OVER') or direc.startswith('LAI2_OVER')]
dataset_list.sort()

In [9]:
data_indices = []
input_images = []
output_labels = []
for DATE in output_2020W_df.index:
    for DIRECTORY in dataset_list:
        if DATE in DIRECTORY:
            file_list = os.listdir(f'./images/2020_W/{DIRECTORY}')
            file_list = [file for file in file_list if file.endswith('.jpg')]
            for FILE in file_list:
                image = Image.open(f'./images/2020_W/{DIRECTORY}/{FILE}')
                data_indices.append(pd.Timestamp(DATE))
                input_images.append(img_to_array(image))
                output_labels.append(output_2020W_df.loc[DATE].values)

In [10]:
input_images = np.stack(input_images, axis=0)
output_labels = np.stack(output_labels, axis=0)
data_indices = np.array(data_indices)

In [11]:
print(len(data_indices))
print(input_images.shape)
print(output_labels.shape)

9335
(9335, 128, 128, 3)
(9335, 4)


In [12]:
# data_indices, input_images, output_labels = resample(data_indices, input_images, output_labels, n_samples=5000, replace=False, random_state=4574)

In [13]:
print(len(data_indices))
print(input_images.shape)
print(output_labels.shape)

9335
(9335, 128, 128, 3)
(9335, 4)


In [14]:
gc.collect()

22

In [15]:
data_indices, input_images, output_labels = shuffle(data_indices, input_images, output_labels, random_state=4574)

In [16]:
N_TRAIN = int(output_labels.shape[0]*.3)
train_input = input_images[:N_TRAIN, ...]
train_label = output_labels[:N_TRAIN, ...]
train_index = data_indices[:N_TRAIN]
test_input = input_images[N_TRAIN:, ...]
test_label = output_labels[N_TRAIN:, ...]
test_index = data_indices[N_TRAIN:]

In [17]:
datagen = ImageDataGenerator(
    rescale=1/255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [18]:
train_iterator = datagen.flow(train_input, train_label, batch_size=BATCH_SIZE)
test_iterator = datagen.flow(test_input, test_label, batch_size=BATCH_SIZE)

In [19]:
gc.collect()

0

# Model construction

In [20]:
with strategy.scope():
    model = Sequential([
        layers.Conv2D(32, 2, padding='same', activation='relu', input_shape=(128, 128, 3)),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 2, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 2, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(512, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(1024, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(4, activation='sigmoid')
        ])

In [21]:
cbs = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=.5, patience=2, verbose=0, mode='min',
    min_delta=0.0001, cooldown=0, min_lr=0)

save = callbacks.ModelCheckpoint(
    BEST_PATH, monitor='val_loss', verbose=0,
    save_best_only=True, save_weights_only=True, mode='min', save_freq='epoch')

early_stop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=20) 

In [22]:
with strategy.scope():
    opt = optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON)
    model.compile(optimizer=opt, loss='mae')
    model.predict(test_iterator)
    model.load_weights(PRE_TRAINED)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


In [23]:
model.trainable = False

In [24]:
with strategy.scope():
    base_model = Sequential(model.layers[:-2])
    prediction_layer1 = layers.Dense(128, activation='relu')
    prediction_layer2 = layers.Dense(4, activation='sigmoid')

In [25]:
with strategy.scope():
    new_model = Sequential([
        base_model,
        prediction_layer1,
        prediction_layer2
    ])

In [26]:
gc.collect()

2500

In [27]:
with strategy.scope():
    opt = optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON)
    new_model.compile(optimizer=opt, loss='mae')
    new_model.fit(train_iterator, epochs=TRAINING_EPOCHS, validation_data=test_iterator,
                  verbose=1, callbacks=[cbs, save, early_stop]) 

Epoch 1/200
INFO:tensorflow:batch_all_reduce: 4 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 4 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
88/88 [==============================] - ETA: 0s - loss: 0.1020INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task

In [28]:
new_model.load_weights(BEST_PATH)

In [29]:
pred_result = model.predict(test_input/255)

In [30]:
test_label = (OUTPUT_MAXS - OUTPUT_MINS)*test_label + OUTPUT_MINS
pred_result = (OUTPUT_MAXS - OUTPUT_MINS)*pred_result + OUTPUT_MINS

In [31]:
pred_df = pd.DataFrame(np.concatenate([test_label, pred_result], axis=1), index=test_index)
pred_df.columns = ['label_node_len', 'label_node_dia', 'label_plant_h', 'label_leaf_area', 'pred_node_len', 'pred_node_dia', 'pred_plant_h', 'pred_leaf_area']

In [32]:
pred_df.to_csv('./results/model_output/pg_transfer_pred_result.csv')